In [1]:
!nvidia-smi

Mon Nov 18 16:36:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   71C    P0             40W /   80W |     457MiB /   8192MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [3]:

import pandas as pd
import tensorflow as tf

import glob
import re

2024-11-18 16:36:23.475889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 16:36:23.537826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 16:36:23.557462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 16:36:23.739932: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
base_path = '/home/workspace/geometry-classifier/data/'



In [5]:
import pandas as pd
import tensorflow as tf
import glob
import os

def serialize_example(name, image_data):
    """Serialize data into TFRecord format."""
    feature = {
        'name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[name])),
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def parquet_to_tfrecord(parquet_file, tfrecord_file):
    """Convert a single Parquet file to a TFRecord file."""
    df = pd.read_csv(parquet_file).drop(columns=["id"])
    
    with tf.io.TFRecordWriter(tfrecord_file, options='GZIP') as writer:
        for _, row in df.iterrows():
            name = row['name'].astype('float32').tobytes()  # Convert uint8 name to bytes
            image_data = row.drop('name').to_numpy().astype('float32').tobytes()  # Flatten image data and convert to bytes
            example = serialize_example(name, image_data)
            writer.write(example)


def convert(file_type:str, files: list):
    i=0
    for parquet_file in files:
        tfrecord_file = f"{base_path}tfrecord2/{file_type}{i}.tfrecord.gz"
        parquet_to_tfrecord(parquet_file, tfrecord_file)
        print(f"Converted {parquet_file} to {tfrecord_file}")
        i+=1


In [7]:
convert('test', glob.glob(f"{base_path}stratified/test*.csv"))


Converted /home/workspace/geometry-classifier/data/stratified/test_150.csv to /home/workspace/geometry-classifier/data/tfrecord2/test0.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/test_201.csv to /home/workspace/geometry-classifier/data/tfrecord2/test1.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/test_135.csv to /home/workspace/geometry-classifier/data/tfrecord2/test2.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/test_174.csv to /home/workspace/geometry-classifier/data/tfrecord2/test3.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/test_155.csv to /home/workspace/geometry-classifier/data/tfrecord2/test4.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/test_8.csv to /home/workspace/geometry-classifier/data/tfrecord2/test5.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/test_146.csv to /home/workspace/geometry-classifier/data/t

In [8]:
convert('val', glob.glob(f"{base_path}stratified/val*.csv"))

Converted /home/workspace/geometry-classifier/data/stratified/val_391.csv to /home/workspace/geometry-classifier/data/tfrecord2/val0.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/val_463.csv to /home/workspace/geometry-classifier/data/tfrecord2/val1.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/val_346.csv to /home/workspace/geometry-classifier/data/tfrecord2/val2.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/val_369.csv to /home/workspace/geometry-classifier/data/tfrecord2/val3.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/val_422.csv to /home/workspace/geometry-classifier/data/tfrecord2/val4.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/val_375.csv to /home/workspace/geometry-classifier/data/tfrecord2/val5.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/val_460.csv to /home/workspace/geometry-classifier/data/tfrecord2/va

In [9]:
train_glob = glob.glob(f"{base_path}stratified/train*.csv")
convert('train', train_glob)


Converted /home/workspace/geometry-classifier/data/stratified/train_667.csv to /home/workspace/geometry-classifier/data/tfrecord2/train0.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_744.csv to /home/workspace/geometry-classifier/data/tfrecord2/train1.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_1205.csv to /home/workspace/geometry-classifier/data/tfrecord2/train2.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_1057.csv to /home/workspace/geometry-classifier/data/tfrecord2/train3.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_1164.csv to /home/workspace/geometry-classifier/data/tfrecord2/train4.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_931.csv to /home/workspace/geometry-classifier/data/tfrecord2/train5.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_902.csv to /home/workspace/geometry